In [ ]:
import numpy as np
from numpy import pi, fft, exp, random

import matplotlib.pyplot as plt


In [ ]:
# define the kernel of square root of covariance
def f(r, sigma, a, dim=1):
    return sigma * (4/a**2/pi)**(dim/4) * exp(-2*r**2 / a**2)

In [ ]:
sigma = 0.01
a = 0.25

L = 1.0 # length of domain
N = 2**9 # number of discretization points
dx = L / N
x = np.arange(-L/2, L/2, dx) # define x domain

# generate GRF path
Num_path = 2**4
sample_u0 = np.zeros((Num_path, N))
for i in range(Num_path):
    w = random.standard_normal(size=(N, )) # difine white noise
    # Generate Gaussian random field with covariance kernel C(r) = f * f (convolution)
    z = fft.ifft(fft.fft(f(r=x, sigma=sigma, a=a, dim=1)) * fft.fft(w))
    z = z.real
    sample_u0[i, :] = z

    plt.plot(x, z) # periodic: z[0] = z[-1] 
    # plt.plot(x, z-z[0]) # z[0] = z[-1] = 0
    plt.xlabel("$x$")
    plt.ylabel("$z$")

plt.grid()
plt.show()

x.shape, z.shape

In [ ]:
# np.savez('./burgers_u0.npz', u0=sample_u0)


In [ ]:
""""
    another way for GRF
    
"""
## define mesh size for x
## finer grid
lx = 1
Nx = 127
dx = lx / (Nx + 1)
points_x = np.linspace(dx, lx - dx, Nx).T
x = points_x[:, None]

## coarser grid
Nx_c = 64
dx_c = lx / (Nx_c + 1)

points_x_c = np.linspace(dx_c, lx - dx_c, Nx_c).T
x_c = points_x_c[:, None]

In [ ]:
# Guassian random field 
Nst = 16
l = 0.5

# kernal matrix
def kernal(xs, ys, l):
    dx = np.expand_dims(xs, 1) - np.expand_dims(ys, 0)
    return np.exp(-((dx / l) ** 2) / 2)


Corv = kernal(points_x, points_x, l)
g_mat = np.zeros((Nst, Nx))
mean = np.zeros_like(points_x)
for i in range(Nst):
    g_mat[[i], :] = np.random.multivariate_normal(mean, Corv)# * points_x * (1 - points_x) # enforce zero B.C.

for i in range(Nst):
    plt.plot(x, g_mat[i, :])

plt.show()

In [ ]:
""""
    another way for GRF
    
"""

import numpy as np
from sklearn import gaussian_process as gp

T = 1.0
num = 1
features = 16
length_scale = 0.5
x = np.linspace(0, T, num=features)[:, None]

def grf(num, features):
    x = np.linspace(0, T, num=features)[:, None]
    K = gp.kernels.RBF(length_scale=length_scale)(x)
    L = np.linalg.cholesky(K + 1e-13 * np.eye(features))
    return (L @ np.random.randn(features, num)).transpose() # (num, features)

plt.plot(x, grf(num, features)[0,:])